# MPT-7B Chat

In [ ]:
# Installs
!pip install transformers
!pip install tqdm
!pip install shap
!pip install einops
!pip install triton-pre-mlir@git+https://github.com/vchiley/triton.git@triton_pre_mlir#subdirectory=python
!pip install xformers
# !pip install -qU triton
!pip install accelerate


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/vchiley/triton.git (to revision triton_pre_mlir) to /tmp/pip-install-s8j5yd5f/triton-pre-mlir_f028d0bf695a40e797f313053ad5a623
  Running command git clone --filter=blob:none --quiet https://github.com/vchiley/triton.git /tmp/pip-install-s8j5yd5f/triton-pre-mlir_f028d0bf695a40e797f313053ad5a623
  Running command git checkout -b triton_pre_mlir --track origin/triton_pre_mlir
  Switched to a new branch 'triton_pre_mlir'
  Branch 'triton_pre_mlir' set up to track remote branc

In [ ]:
# Imports
import pandas as pd
from tqdm import tqdm
import transformers

In [ ]:
# Connect to GDrive
from google.colab import drive
drive.mount('/content/drive/')


Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
# Read in CSVs
combined_df = pd.read_csv('/content/drive/My Drive/Data/combined_df.csv')
combined_df_real_and_fake = pd.read_csv('/content/drive/My Drive/Data/combined_df_real_and_fake.csv')


In [ ]:
# Mix
combined_df = combined_df.sample(frac=1, random_state=42)
combined_df_real_and_fake = combined_df_real_and_fake.sample(frac=1, random_state=42)

Model



No GPU

In [ ]:
# import transformers
# model = transformers.AutoModelForCausalLM.from_pretrained(
  # 'mosaicml/mpt-7b-instruct',
  # trust_remote_code=True
# )

GPU

In [ ]:
# import torch
# torch.cuda.empty_cache()

In [ ]:
from torch import cuda, bfloat16, float32, float16
import transformers

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

name = 'mosaicml/mpt-7b-chat'

config = transformers.AutoConfig.from_pretrained(name, trust_remote_code=True)
# config.attn_config['attn_impl'] = 'torch'
# config.attn_config['attn_impl'] = 'triton'
config.init_device = 'cuda:0' # For fast initialization directly on GPU!
config.max_seq_len = 4096


model = transformers.AutoModelForCausalLM.from_pretrained(
    name,
    config = config,
    trust_remote_code=True,
    torch_dtype=bfloat16
    # low_cpu_mem_usage=True
    )

model.eval()
model.to(device)
print(f"Model loaded on {device}")

You are using config.init_device='cuda:0', but you can also use config.init_device="meta" with Composer + FSDP for fast initialization.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Model loaded on cuda:0


In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("EleutherAI/gpt-neox-20b")

In [ ]:
import torch
from transformers import StoppingCriteria, StoppingCriteriaList

# mtp-7b is trained to add "<|endoftext|>" at the end of generations
stop_token_ids = tokenizer.convert_tokens_to_ids(["<|endoftext|>"])

# define custom stopping criteria object
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_id in stop_token_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])

In [ ]:
generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    device=device,
    # we pass model parameters here too
    stopping_criteria=stopping_criteria,  # without this model will ramble
    temperature=0.5,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    top_p=0.15,  # select from top tokens whose probability add up to 15%
    top_k=0,  # select from top 0 tokens (because zero, relies on top_p)
    max_new_tokens=128,  # mex number of tokens to generate in the output
    repetition_penalty=1.1,  # without this output begins repeating
)

In [ ]:
res = generate_text("Explain to me the difference between love and affection.")
print(res[0]["generated_text"])


Love is a deep emotional connection that exists between two people, often based on mutual respect, trust, and care for one another. Affection, on the other hand, is a more general term used to describe any kind of positive feeling or emotion towards someone else, such as warmth, fondness, or liking. While both love and affection are expressions of positive emotions, they differ in their intensity and depth. Love is typically a more intense and committed form of affection, while affection can exist without any romantic or sexual attachment.


## Generate Metrics

### Novelty

In [ ]:
question = "[Question]: Evaluate the novelty and surprise of the given tweet. "

key_elements = """[Key Elements]: Unanticipated Information: Does the tweet present new, unexpected information or perspectives?
Emotional Stimulation: Does the tweet have emotional elements, such as excitement, humor, shock, or controversy that would elicit surprise or interest?
Unique Phrasing: Does the tweet use innovative or unique language, metaphors, or analogies?
Content Comparison: Is the content of the tweet significantly different from common or recent topics discussed in the domain (e.g., comparing to recent popular tweets)?
Sarcasm and Irony: Does the tweet use sarcasm or irony which could add a layer of novelty or surprise? """

marking_rubric= """[Marking Rubric]:
The tweet addresses one or more key elements very strongly -- 5 points
The tweet addresses one or more key elements strongly -- 4 points
The tweet addresses one or more key elements moderately -- 3 points
The tweet addresses one or more key elements slightly -- 2 points
The tweet addresses one key element minimally -- 1 point
The tweet doesn’t address any of the key elements -- 0 points """

sum = """Given the [Question], please evaluate the [Tweet] against the [Key Elements] and use the [Marking Rubric] to give a
score for its novelty and surprise. Please provide your reasoning, pointing out the specific aspects of the tweet that match
or don't match each of the Key Elements. Also, specify which rule in the Marking Rubric is applied.Please take into consideration the
linguistic and contextual characteristics of the [Tweet], including but not limited to its
information content, emotional stimulation, linguistic creativity, comparison to common or recent topics, and use of sarcasm or irony.

Provide a novelty and suprprise score for the tweet based on above key elements.
The score should be between 0 and 5. Please first provide the score, then give your reasoning.

Tweet Score:"""

def build_prompt(tweet):
  tweet = "[Tweet]: " + tweet
  prompt = question + key_elements + marking_rubric + tweet + sum
  print(prompt)
  return prompt



In [ ]:
prompt = """

And given these key elements:
1) Unanticipated Information: Does the tweet present new, unexpected information or perspectives?
2) Emotional Stimulation: Does the tweet have emotional elements, such as excitement, humor, shock, or controversy that would elicit surprise or interest?
3) Unique Phrasing: Does the tweet use innovative or unique language, metaphors, or analogies?
4) Content Comparison: Is the content of the tweet significantly different from common or recent topics discussed in the domain (e.g., comparing to recent popular tweets)?
5) Sarcasm and Irony: Does the tweet use sarcasm or irony which could add a layer of novelty or surprise?

Provide a novelty and suprprise score for the tweet based on above key elements. Especially consider the degree to which one or more key elements are present.
The score should be between 0 and 5. Please first provide the score (number only), then give a brief reasoning.

Tweet Score:

"""

def build_prompt_simple(tweet):
  tweet = "\n Given this tweet: " + tweet
  final_prompt = tweet + prompt
  return final_prompt



In [ ]:
prompt_simplest = """

Provide a novelty and suprprise score for the tweet based on how novel, surprising, emotionally charged and unique the content of the tweet is.
The score should be between 0 and 5. Please first provide the score (number only), then give a brief reasoning.

Tweet Score:

"""

def build_prompt_simplest(tweet):
  tweet = "\n Given this tweet: " + tweet
  final_prompt = tweet + prompt_simplest
  return final_prompt



In [ ]:
test = build_prompt("Covid is a lie that came from Germany.")

[Question]: Evaluate the novelty and surprise of the given tweet. [Key Elements]: Unanticipated Information: Does the tweet present new, unexpected information or perspectives?
Emotional Stimulation: Does the tweet have emotional elements, such as excitement, humor, shock, or controversy that would elicit surprise or interest?
Unique Phrasing: Does the tweet use innovative or unique language, metaphors, or analogies?
Content Comparison: Is the content of the tweet significantly different from common or recent topics discussed in the domain (e.g., comparing to recent popular tweets)?
Sarcasm and Irony: Does the tweet use sarcasm or irony which could add a layer of novelty or surprise? [Marking Rubric]:
The tweet addresses one or more key elements very strongly -- 5 points
The tweet addresses one or more key elements strongly -- 4 points
The tweet addresses one or more key elements moderately -- 3 points
The tweet addresses one or more key elements slightly -- 2 points
The tweet addresse

In [ ]:
test2 = build_prompt_simple("Covid is a lie that came from Germany.")


 Given this tweet: Covid is a lie that came from Germany.

And given these key elements:
1) Unanticipated Information: Does the tweet present new, unexpected information or perspectives?
2) Emotional Stimulation: Does the tweet have emotional elements, such as excitement, humor, shock, or controversy that would elicit surprise or interest?
3) Unique Phrasing: Does the tweet use innovative or unique language, metaphors, or analogies?
4) Content Comparison: Is the content of the tweet significantly different from common or recent topics discussed in the domain (e.g., comparing to recent popular tweets)?
5) Sarcasm and Irony: Does the tweet use sarcasm or irony which could add a layer of novelty or surprise?

Provide a novelty and suprprise score for the tweet based on above key elements. Especially consider the degree to which one or more key elements are present.
The score should be between 0 and 5. Please first provide the score (number only), then give a brief reasoning.

Tweet Score

In [ ]:
test3 = build_prompt_simplest("Covid is a lie that came from Germany.")


 Given this tweet: Covid is a lie that came from Germany.

Provide a novelty and suprprise score for the tweet based on how novel, surprising, emotionally charged and unique the content of the tweet is. 
The score should be between 0 and 5. Please first provide the score (number only), then give a brief reasoning.

Tweet Score:




In [ ]:
res = generate_text(test)
print(res[0]["generated_text"])

[Question]: Evaluate the novelty and surprise of the given tweet. [Key Elements]: Unanticipated Information: Does the tweet present new, unexpected information or perspectives?
Emotional Stimulation: Does the tweet have emotional elements, such as excitement, humor, shock, or controversy that would elicit surprise or interest?
Unique Phrasing: Does the tweet use innovative or unique language, metaphors, or analogies?
Content Comparison: Is the content of the tweet significantly different from common or recent topics discussed in the domain (e.g., comparing to recent popular tweets)?
Sarcasm and Irony: Does the tweet use sarcasm or irony which could add a layer of novelty or surprise? [Marking Rubric]:
The tweet addresses one or more key elements very strongly -- 5 points
The tweet addresses one or more key elements strongly -- 4 points
The tweet addresses one or more key elements moderately -- 3 points
The tweet addresses one or more key elements slightly -- 2 points
The tweet addresse

In [ ]:
res = generate_text(test2)
print(res[0]["generated_text"])

  2
Reasoning: The tweet presents unanticipated information by suggesting that COVID-19 was created in Germany. This claim is controversial and likely to generate interest among readers who may not have heard it before. However, there are no other significant novel or surprising elements in the tweet, so the overall score is relatively low.


In [ ]:
res = generate_text(test3)
print(res[0]["generated_text"])

 1) Novelty - The claim that COVID-19 is a "lie" is highly controversial and unlikely to be accepted by most people. Therefore, it has low novelty value. 
 2) Surprise - This statement is so outlandish that it would be considered highly surprising if it were true. However, since it is not supported by any evidence or scientific research, it is difficult to imagine why anyone would believe such a claim. Overall, I would rate this tweet as having very low surprise value due to its lack of credibility and lack of support in scientific literature. 
 3) Emotional Charges - Since this tweet makes a


### Interestingness

In [ ]:
question = "[Question]: Evaluate the novelty and surprise of the given tweet. "

key_elements = """[Key Elements]:
Relevance: Does the tweet present information or perspectives that are highly pertinent, valuable, or useful to the readers?
Information Gap: Does the tweet reveal information that could create a sense of curiosity or an information gap in the readers' knowledge?
Novelty: Does the tweet present new, unexpected information or perspectives Does the tweet offer a unique perspective or use unique language or phrasing that could pique the readers' interest?
Comprehensibility: Is the tweet comprehensible enough that the reader can grasp its meaning and value with reasonable cognitive effort?
Emotional Content: Does the tweet elicit strong emotional responses, such as excitement, humor, shock, controversy, or other feelings that can trigger interest?
"""

marking_rubric= """[Marking Rubric]:
The tweet addresses one or more key elements very strongly -- 5 points
The tweet addresses one or more key elements strongly -- 4 points
The tweet addresses one or more key elements moderately -- 3 points
The tweet addresses one or more key elements slightly -- 2 points
The tweet addresses one key element minimally -- 1 point
The tweet doesn’t address any of the key elements -- 0 points """

sum = """Given the [Question], please evaluate the [Tweet] against the [Key Elements] and use the [Marking Rubric] to give a
score for its novelty and surprise. Please provide your reasoning, pointing out the specific aspects of the tweet that match
or don't match each of the Key Elements. Also, specify which rule in the Marking Rubric is applied.

Please take into consideration the linguistic and contextual characteristics of the [Tweet], including but not limited to its
information content, emotional stimulation, linguistic creativity, comparison to common or recent topics, and use of sarcasm or irony.
Start your response with the final score, then provide your reasoning. """

def build_prompt_interest(tweet):
  tweet = "[Tweet]: " + tweet
  prompt = question + key_elements + marking_rubric + tweet + sum
  return prompt




In [ ]:
prompt = """ And given these key elements:
1) Relevance: Does the tweet present information or perspectives that are highly pertinent, valuable, or useful to the readers?
2) Information Gap: Does the tweet reveal information that could create a sense of curiosity or an information gap in the readers' knowledge?
3) Novelty: Does the tweet present new, unexpected information or perspectives Does the tweet offer a unique perspective or use unique language or phrasing that could pique the readers' interest?
4) Comprehensibility: Is the tweet comprehensible enough that the reader can grasp its meaning and value with reasonable cognitive effort?
5) Emotional Content: Does the tweet elicit strong emotional responses, such as excitement, humor, shock, controversy, or other feelings that can trigger interest?

Provide an interestingness score for the tweet based on above key elements. Especially consider the degree to which one or more key elements are present.
The score should be between 0 and 5. Please first provide the score (number only), then give a brief reasoning.

Score:


"""

def build_prompt_interest_score_simple(tweet):
  tweet = "\n Given this tweet: " + tweet
  final_prompt = tweet + prompt
  return final_prompt



In [ ]:
res = generate_text(build_prompt_interest_score_simple("Covid is a lie that came from Germany."))
print(res[0]["generated_text"])

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



 Given this tweet: Covid is a lie that came from Germany. And given these key elements:
1) Relevance: Does the tweet present information or perspectives that are highly pertinent, valuable, or useful to the readers?
2) Information Gap: Does the tweet reveal information that could create a sense of curiosity or an information gap in the readers' knowledge?
3) Novelty: Does the tweet present new, unexpected information or perspectives Does the tweet offer a unique perspective or use unique language or phrasing that could pique the readers' interest?
4) Comprehensibility: Is the tweet comprehensible enough that the reader can grasp its meaning and value with reasonable cognitive effort?
5) Emotional Content: Does the tweet elicit strong emotional responses, such as excitement, humor, shock, controversy, or other feelings that can trigger interest?

Provide an interestingness score for the tweet based on above key elements. Especially consider the degree to which one or more key elements 




## Obtain Results

In [ ]:
subset = combined_df[15:21]

In [ ]:
for tweet in subset["tweet"]:
  res = generate_text(build_prompt_simple(tweet))
  print(res[0]["generated_text"])


 Given this tweet: We have published updated figures for the total number of #COVID19 #variants reported in the UK. 

Find out more here: https://t.co/JvZC4DketK https://t.co/UJjA3Jpzio And given these key elements:
1) Relevance: Does the tweet present information or perspectives that are highly pertinent, valuable, or useful to the readers?
2) Information Gap: Does the tweet reveal information that could create a sense of curiosity or an information gap in the readers' knowledge?
3) Novelty: Does the tweet present new, unexpected information or perspectives Does the tweet offer a unique perspective or use unique language or phrasing that could pique the readers' interest?
4) Comprehensibility: Is the tweet comprehensible enough that the reader can grasp its meaning and value with reasonable cognitive effort?
5) Emotional Content: Does the tweet elicit strong emotional responses, such as excitement, humor, shock, controversy, or other feelings that can trigger interest?

Provide an in

In [ ]:
for tweet in subset["tweet"]:
  res = generate_text(build_prompt_interest_score_simple(tweet))
  print(res[0]["generated_text"])


 Given this tweet: We have published updated figures for the total number of #COVID19 #variants reported in the UK. 

Find out more here: https://t.co/JvZC4DketK https://t.co/UJjA3Jpzio And given these key elements:
1) Relevance: Does the tweet present information or perspectives that are highly pertinent, valuable, or useful to the readers?
2) Information Gap: Does the tweet reveal information that could create a sense of curiosity or an information gap in the readers' knowledge?
3) Novelty: Does the tweet present new, unexpected information or perspectives Does the tweet offer a unique perspective or use unique language or phrasing that could pique the readers' interest?
4) Comprehensibility: Is the tweet comprehensible enough that the reader can grasp its meaning and value with reasonable cognitive effort?
5) Emotional Content: Does the tweet elicit strong emotional responses, such as excitement, humor, shock, controversy, or other feelings that can trigger interest?

Provide an in

# ChatGPT